In [0]:
%run "../includes/configuration"

In [0]:
display(dbutils.fs.mounts())

In [0]:
%fs
ls /mnt/f1dlnagsa/raw

## Circuits table

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
circuit_schema = StructType([
    StructField("circuitId", IntegerType(), False),
    StructField("circuitRef", StringType(), False),
    StructField("name", StringType(), False),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", FloatType(), True),
    StructField("lng", FloatType(), True),
    StructField("alt", IntegerType(), True),
    StructField("url", StringType(), False)
])

In [0]:
circuits_df = spark.read.csv("dbfs:/mnt/f1dlnagsa/raw/circuits.csv", header=True, schema=circuit_schema)

In [0]:
from pyspark.sql.functions import col
circuits_selected_df = circuits_df.select(col("circuitId").alias("circuit_id"), col("circuitRef").alias("circuit_ref"), col("name"), col("location"), col("country"), col("lat").alias("latitude"), col("lng").alias("longitude"), col("alt").alias("altitude"))

In [0]:
#Adding ingestion date to keep track of recently updated data
from pyspark.sql.functions import current_timestamp
circuits_final_df = circuits_selected_df.withColumn("ingestion_date", current_timestamp())

Write data to /processed file as parquet

In [0]:
circuits_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.circuits")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/circuits"))

In [0]:
%sql
SELECT * FROM f1_processed.circuits;